# import small dataset  

In [1]:
import pandas as pd
import datetime

%matplotlib inline

In [ ]:
master_df = pd.read_csv('/database/small_master.csv')

In [ ]:
forecast_df = pd.read_csv('/database/small_forecast.csv')

# pre-processing dataset

In [71]:
def pre_master_data(master_df):
# Load and copy data
    df = test_master.copy()
# cleaning and processing data
    df.rename(columns={'timestamp_eastern': 'ts'}, inplace=True)
    temp = pd.DatetimeIndex(df['ts'])
    df['ts'] = pd.to_datetime(df['ts'])
    cols_to_keep = ['season','adjusted_demand_MW','demand_MW','hour_ending_eastern','ts']
    df = df[cols_to_keep]
    df['ts'] = pd.to_datetime(df['ts'])
    df['rankings_per_day'] = df.groupby(['season', df.ts.dt.date]).demand_MW.rank(ascending=False)
    df = df[df['rankings_per_day'] == 1]
    df.reset_index(drop=True, inplace=True)
# Group by season and rank by adjusted_demand_MW
    df['rankings_per_season'] = df.groupby(['season']).demand_MW.rank(ascending=False)
    return df

In [72]:
pre_master_data(master_df)

,season,adjusted_demand_MW,demand_MW,hour_ending_eastern,ts,rankings_per_day,rankings_per_season
0,2016-2017,14831.0,15083.0,20,2016-05-01 20:00:00,1.0,340.0
1,2016-2017,15622.0,16116.0,21,2016-05-02 21:00:00,1.0,286.0
2,2016-2017,15465.0,15855.0,20,2016-05-03 20:00:00,1.0,304.0
3,2016-2017,15450.0,15884.0,20,2016-05-04 20:00:00,1.0,302.5
4,2016-2017,15284.0,15809.0,20,2016-05-05 20:00:00,1.0,309.0
...,...,...,...,...,...,...,...
360,2016-2017,15158.0,15709.0,19,2017-04-27 19:00:00,1.0,319.0
361,2016-2017,14739.0,15008.0,20,2017-04-28 20:00:00,1.0,344.0
362,2016-2017,13697.0,13855.0,21,2017-04-29 21:00:00,1.0,360.0
363,2016-2017,14539.0,14891.0,20,2017-04-30 20:00:00,1.0,347.0


In [116]:
def pre_forecast_data(master_df):
    df3 = test_forecast.copy()
    temp = pd.DatetimeIndex(df3['ts'])
    df3['ts'] = pd.to_datetime(df3['ts'])
    df3['ts_future'] = pd.to_datetime(df3['ts_future'])
    cols_to_keep = ['ts', 'ts_future','forecast']
    df3 = df3[cols_to_keep]
    df_ten = df3[(df3.ts.dt.time == datetime.time(10,0))]
    df_ten = df_ten[['forecast','ts','ts_future']]
    df_ten.rename(columns={'ts': 'ts_main'}, inplace=True)
    df_ten.rename(columns={'ts_future': 'ts'}, inplace=True)
    df_ten = df_ten[['forecast','ts']]
    return df_ten

In [117]:
pre_forecast_dataset(master_df)

,forecast,ts
216,0.0025,2016-05-08 11:00:00
217,0.0000,2016-05-08 12:00:00
218,0.0050,2016-05-08 13:00:00
219,0.0000,2016-05-08 14:00:00
220,0.0000,2016-05-08 15:00:00
...,...,...
199531,0.0000,2017-04-20 06:00:00
199532,0.0000,2017-04-20 07:00:00
199533,0.0000,2017-04-20 08:00:00
199534,0.0775,2017-04-20 09:00:00


# discharge strategy

### choose top n in small_forecast file (top 12 per day)
### for example, if we choose top3 largest in small_forecast file, giving 0.6 0.2 0.1. 
### Our strategy is to sum all the number which is 0.9 and then we will equally distributed 0.1. 
### Right now, we will get the probabilities 0.633, 0.233, 0.133. 
### If a probability is greater than 0.5, you can still only discharge the battery at its max rating (0.5), the rest of the probabilities we will discharge it proportional.


In [75]:
def top_forecast(ts,n, forecast_df):
    df3 = forecast_df.copy()
    df3['ts'] = pd.to_datetime(df3['ts'])
    df3['ts_future'] = pd.to_datetime(df3['ts_future'])
    cols_to_keep = ['ts', 'ts_future', 'forecast']
    df3 = df3[cols_to_keep]
    df_ten = df3[(df3.ts.dt.time == datetime.time(10, 0))]
    df_ten = df_ten[['forecast', 'ts', 'ts_future']]
    df_ten2 = df_ten[(df_ten['ts_future'] == ts)]
    ts_10 = df_ten2['ts'].max()
    df_ten = df_ten[(df_ten['ts'] == ts_10)]
    nlar = df_ten.nlargest(n, 'forecast')
    df = nlar['forecast']
    df_s = df.sum()
    dff = (1-df_s)/n
    nlar['pro'] = nlar['forecast'].apply(lambda x: (x + dff))
    result = nlar[(nlar['ts_future'] == ts)]
    if len(result['pro']) == 0:
        return 0
    df_p = result['pro'].max()
    if df_p >= 0.5:
        return (0.5/0.5)*100
    else:
        p = int((df_p/0.5)*100)
        return p

In [76]:
top_forecast('2019-07-19 12:00:00',12)

0

In [77]:
def merge_forecast_top_priority(master_df,forecast_df):
    dff = pre_master_dataset(master_df) 
    dff2 = pre_forecast_dataset(forecast_df)
    df_merge = dff.merge(dff2,on='ts')
    df_merge['predict']=(df_merge.forecast>0).astype(int)
    return df_merge

In [78]:
merge_forecast_top_priority(master_df,forecast_df)

,season,adjusted_demand_MW,demand_MW,hour_ending_eastern,ts,rankings_per_day,rankings_per_season,forecast,predict
0,2016-2017,13874.0,14361.0,21,2016-05-08 21:00:00,1.0,354.0,0.3075,1
1,2016-2017,15343.0,15818.0,20,2016-05-09 20:00:00,1.0,308.0,0.4200,1
2,2016-2017,15185.0,15570.0,21,2016-05-10 21:00:00,1.0,324.0,0.5850,1
3,2016-2017,15216.0,15663.0,21,2016-05-11 21:00:00,1.0,322.0,0.7350,1
4,2016-2017,15561.0,15884.0,20,2016-05-12 20:00:00,1.0,302.5,0.1700,1
...,...,...,...,...,...,...,...,...,...
341,2016-2017,13481.0,13598.0,20,2017-04-15 20:00:00,1.0,363.0,0.3775,1
342,2016-2017,13364.0,13650.0,20,2017-04-16 20:00:00,1.0,362.0,0.4000,1
343,2016-2017,15613.0,15979.0,20,2017-04-17 20:00:00,1.0,296.0,0.6675,1
344,2016-2017,15726.0,15762.0,20,2017-04-18 20:00:00,1.0,316.5,0.6800,1


In [185]:
def accuracy(master_df,forecast_df):
    num = [1, 3, 5, 10, 20]
    new = []
    data = merge_forecast_top_priority(master_df, forecast_df)
    df = data.copy()
    for i in df.groupby('season'):
        for j in num:
            tmp = df[(df['season'] == i[0]) & (df['rankings_per_season'] <= j)]
            test = tmp.forecast.apply(lambda x: (x > 0))
            df = df[df['rankings_per_season'] <= 20]
            dff_uni = pd.unique(df['hour_ending_eastern'])
            count = 0
            for k, v in test.items():
                if v is True:
                    count += 1
            row = {
                'season': i[0],
                'top_n': j,
                'success': count,
                'Hit rate': count / j,
            }
            performance = 0
            for peak_hr in dff_uni:
                df_peak_hr = tmp[tmp['hour_ending_eastern'] == peak_hr]
                sum = 0
                ct = 0
                for z in df_peak_hr['ts']:
                    ct += 1
                    sum += top_forecast(z, 3, forecast_df)
                ave = 0
                if ct != 0:
                    ave = int(sum / ct)
                performance += int(ave * df_peak_hr.predict.sum())
                if ave == 0:
                    row[peak_hr] = f'{0}/{df_peak_hr.predict.count()},{ave}'
                    row['success'] = row['success'] - df_peak_hr.predict.count()
                    row['Hit rate'] = row['success'] / j
                else:
                    row[peak_hr] = f'{df_peak_hr.predict.sum()}/{df_peak_hr.predict.count()},{ave}'
            row['performance'] = performance/j
            new.append(row)
        final_df = pd.DataFrame(new)
    return final_df

In [149]:
accuracy(master_df,forecast_df)

2016-09-07 17:00:00
2016-08-10 18:00:00
2016-08-11 17:00:00
2016-09-07 17:00:00
2016-07-13 18:00:00
2016-08-10 18:00:00
2016-08-11 17:00:00
2016-08-12 17:00:00
2016-09-07 17:00:00
2016-07-13 18:00:00
2016-08-10 18:00:00
2016-09-08 18:00:00
2016-07-22 17:00:00
2016-08-04 17:00:00
2016-08-11 17:00:00
2016-08-12 17:00:00
2016-09-06 17:00:00
2016-09-07 17:00:00
2016-08-05 13:00:00
2016-06-20 18:00:00
2016-07-13 18:00:00
2016-08-10 18:00:00
2016-09-08 18:00:00
2016-06-27 17:00:00
2016-07-21 17:00:00
2016-07-22 17:00:00
2016-07-26 17:00:00
2016-08-03 17:00:00
2016-08-04 17:00:00
2016-08-11 17:00:00
2016-08-12 17:00:00
2016-08-19 17:00:00
2016-09-06 17:00:00
2016-09-07 17:00:00
2016-07-12 19:00:00
2016-07-27 16:00:00
2016-08-09 16:00:00
2016-08-25 16:00:00
2016-08-05 13:00:00


,season,year,top_n,success,Hit rate,18,17,19,16,13,percentage,quality
0,2016-2017,2016-2017,1,1,1.0,"0/0,0","1/1,100","0/0,0","0/0,0","0/0,0",100,good
1,2016-2017,2016-2017,3,3,1.0,"1/1,44","2/2,100","0/0,0","0/0,0","0/0,0",81,good
2,2016-2017,2016-2017,5,5,1.0,"2/2,46","3/3,100","0/0,0","0/0,0","0/0,0",78,good
3,2016-2017,2016-2017,10,10,1.0,"3/3,42","6/6,100","0/0,0","0/0,0","1/1,32",75,good
4,2016-2017,2016-2017,20,20,1.0,"4/4,42","11/11,100","1/1,0","3/3,0","1/1,32",65,poor
